# Q4

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
class tanh:
    
    def __init__(self):
        self.inputs = None
        self.outputs = None
        
    def forward_pass(self, inputs):
        self.inputs = inputs
        return np.tanh(inputs)
    
    def backward_pass(self, output_gradient):
        return np.multiply(output_gradient, (1 - np.square(np.tanh(self.inputs))))

class sigmoid:
    
    def __init__(self):
        self.inputs = None
        self.outputs = None
        
    def forward_pass(self, inputs):
        self.inputs = inputs
        return np.reciprocal(1 + np.exp(-1 * inputs))
    
    def backward_pass(self, output_gradient):
        return np.multiply(output_gradient, np.multiply(np.reciprocal(1 + np.exp(-1 * self.inputs)), (1 - np.reciprocal(1 + np.exp(-1 * self.inputs)))))

class softmax:
    
    def __init__(self):
        self.inputs = None
        self.outputs = None
        
    def forward_pass(self, inputs):
        s = np.sum(np.exp(inputs))
        self.outputs = np.exp(inputs) / s
        return self.outputs
    
    def backward_pass(self, output_gradient):
        n = np.size(output_gradient)
        I = np.matrix(np.ones(n)).transpose()
        return np.multiply(np.multiply(self.outputs, I - self.outputs), output_gradient)
    
class linear:
    
    def __init__(self):
        self.inputs = None
        self.outputs = None
    
    def forward_pass(self, inputs):
        return inputs
    
    def backward_pass(self, output_gradient):
        return output_gradient
    
class mse_loss:
    
    def __init__(self):
        self.Y_pred = None
        self.outputs = None
    
    def forward_pass(self, Y_pred, Y):
        self.Y_pred = Y_pred
        self.Y = Y
        #print(Y_pred.shape, Y.shape)
        return np.sum(np.square(Y - Y_pred))/np.size(Y)
    
    def backward_pass(self):
        out = 2 * (self.Y_pred - self.Y) / np.size(self.Y)
        #print(out.shape)
        return out
        
class cross_entropy_loss:
    
    def __init__(self):
        self.Y_pred = None
        self.outputs = None
        
    def forward_pass(self, Y_pred, Y):
        self.Y_pred = Y_pred
        self.Y = Y
        n = np.size(self.Y)
        I = np.matrix(np.ones(n)).transpose()
        return -np.sum(np.multiply(Y, np.log(Y_pred)) + np.multiply(I - Y, np.log(I - Y_pred)))
    
    def backward_pass(self):
        n = np.size(self.Y)
        I = np.matrix(np.ones(n)).transpose()
        return (I - self.Y) / (I - self.Y_pred) - self.Y / self.Y_pred
    
    
class makelayer:
    
    def __init__(self, in_neuron, out_neuron, activation):
        self.activation = activation
        self.in_neuron= in_neuron
        self.out_neuron= out_neuron
        self.w = np.matrix(np.random.randn(out_neuron, in_neuron))
        self.b = np.matrix(np.random.randn(out_neuron)).transpose()

    def forward_pass(self, inputs):
        self.inputs = inputs
        outputs = self.w @ inputs + self.b
        return self.activation.forward_pass(outputs)
    
    def backward_pass(self, output_gradient, learning_rate = 0.01):
        activation_gradient = self.activation.backward_pass(output_gradient)
        w_gradient = activation_gradient @ self.inputs.transpose()
        b_gradient = activation_gradient
        self.w -= learning_rate * w_gradient
        self.b -= learning_rate * b_gradient
        input_gradient = self.w.transpose() @ activation_gradient
        return input_gradient
        

class NeuralNetwork:
    
    def __init__(self, X, Y, list_layers, loss, iteration = 2001, learning_rate = 0.01):
        self.X = X
        self.Y = Y
        self.features = X[0].size
        self.list_layers = list_layers
        self.loss = loss
        self.iteration = iteration
        self.learning_rate = learning_rate
    
    def train(self):
        self.train_layers_list = []
        prev = self.features
        for (output_neurons, activation) in self.list_layers:
            self.train_layers_list.append(makelayer(prev, output_neurons, activation))
            prev = output_neurons

        for iter in range(self.iteration):
            if iter%100==0:
                print("iterations completed:",iter)
            for x, y in zip(self.X, self.Y):
                #print(x,y)
                y = y.transpose()
                output = x.transpose()
                #print(x,x.shape, output.shape)
                for layer in self.train_layers_list:
                    output = layer.forward_pass(output)
                output = self.loss.forward_pass(output, y)

                grad = self.loss.backward_pass()
                #print(grad, grad.shape)
                for layer in reversed(self.train_layers_list):
                    grad = layer.backward_pass(grad, self.learning_rate)

    def test(self, X_test):
        outputs = []
        for x in X_test:
            output = x.transpose()
            for layer in self.train_layers_list:
                output = layer.forward_pass(output)
            outputs.append(output.T.tolist()[0])
        return outputs

# Q5

In [4]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()

In [5]:
X = boston_dataset.data
Y = boston_dataset.target
X = (X - X.min()) / (X.max() - X.min())
Y = (Y - Y.min()) / (Y.max() - Y.min())
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [6]:
X_train = np.matrix(X_train)
X_test = np.matrix(X_test)
Y_train = np.matrix(Y_train).T
Y_test = np.matrix(Y_test).T

## a

In [7]:
layers1 = [(1, linear())]
NN1 = NeuralNetwork(X_train, Y_train, layers1, mse_loss())
NN1.train()
pred1 = np.matrix(NN1.test(X_test))

iterations completed: 0
iterations completed: 100
iterations completed: 200
iterations completed: 300
iterations completed: 400
iterations completed: 500
iterations completed: 600
iterations completed: 700
iterations completed: 800
iterations completed: 900
iterations completed: 1000
iterations completed: 1100
iterations completed: 1200
iterations completed: 1300
iterations completed: 1400
iterations completed: 1500
iterations completed: 1600
iterations completed: 1700
iterations completed: 1800
iterations completed: 1900
iterations completed: 2000


In [8]:
mse1 = np.sum(np.square(Y_test - pred1))/Y_test.size
p_error1 = 0
for i in range(Y_test.size):
    temp = Y_test.item(i)
    if temp != 0:
        p_error1 += abs(temp - pred1.item(i)) / temp
p_error1 *= 100 / Y_test.size
print("Mean Squared Error:",mse1,"\nPercentage Error:",p_error1,"%")

Mean Squared Error: 0.01608347590587703 
Percentage Error: 31.808769139208888 %


## b

In [9]:
layers2 = [(13, sigmoid()), (1, linear())]
NN2 = NeuralNetwork(X_train, Y_train, layers2, mse_loss())
NN2.train()
pred2 = np.matrix(NN2.test(X_test))

iterations completed: 0
iterations completed: 100
iterations completed: 200
iterations completed: 300
iterations completed: 400
iterations completed: 500
iterations completed: 600
iterations completed: 700
iterations completed: 800
iterations completed: 900
iterations completed: 1000
iterations completed: 1100
iterations completed: 1200
iterations completed: 1300
iterations completed: 1400
iterations completed: 1500
iterations completed: 1600
iterations completed: 1700
iterations completed: 1800
iterations completed: 1900
iterations completed: 2000


In [10]:
mse2 = np.sum(np.square(Y_test - pred2))/Y_test.size
p_error2 = 0
for i in range(Y_test.size):
    temp = Y_test.item(i)
    if temp != 0:
        p_error2 += abs(temp - pred2.item(i)) / temp
p_error2 *= 100 / Y_test.size
print("Mean Squared Error:",mse2,"\nPercentage Error:",p_error2,"%")

Mean Squared Error: 0.015558959254216262 
Percentage Error: 27.12910832380324 %


## c

In [11]:
layers3 = [(13, sigmoid()), (13, sigmoid()), (1, linear())]
NN3 = NeuralNetwork(X_train, Y_train, layers3, mse_loss())
NN3.train()
pred3 = np.matrix(NN3.test(X_test))

iterations completed: 0
iterations completed: 100
iterations completed: 200
iterations completed: 300
iterations completed: 400
iterations completed: 500
iterations completed: 600
iterations completed: 700
iterations completed: 800
iterations completed: 900
iterations completed: 1000
iterations completed: 1100
iterations completed: 1200
iterations completed: 1300
iterations completed: 1400
iterations completed: 1500
iterations completed: 1600
iterations completed: 1700
iterations completed: 1800
iterations completed: 1900
iterations completed: 2000


In [12]:
mse3 = np.sum(np.square(Y_test - pred3))/Y_test.size
p_error3 = 0
for i in range(Y_test.size):
    temp = Y_test.item(i)
    if temp != 0:
        p_error3 += abs(temp - pred3.item(i)) / temp
p_error3 *= 100 / Y_test.size
print("Mean Squared Error:",mse3,"\nPercentage Error:",p_error3,"%")

Mean Squared Error: 0.015799280013729128 
Percentage Error: 26.835279247506946 %


# 6

In [13]:
from sklearn.datasets import load_digits
mnist_dataset = load_digits()

In [14]:
X = mnist_dataset.data
Y = mnist_dataset.target
X = X / 255
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [15]:
new_Y = np.zeros((Y_train.size,10))
for i in range(Y_train.size):
    new_Y[i][Y_train[i]] = 1
Y_train = new_Y
X_train = np.matrix(X_train)
X_test = np.matrix(X_test)
Y_train = np.matrix(Y_train)
Y_test = np.matrix(Y_test)

## a 

In [17]:
layers4 = [(89, tanh()), (10, sigmoid())]
NN4 = NeuralNetwork(X_train, Y_train, layers4, mse_loss(), iteration = 801)
NN4.train()
pred4 = NN4.test(X_test)

iterations completed: 0
iterations completed: 100
iterations completed: 200
iterations completed: 300
iterations completed: 400
iterations completed: 500
iterations completed: 600
iterations completed: 700
iterations completed: 800


In [18]:
correct_num = Y_test.tolist()[0]
pred_num = []
correct_pred = 0
for i in range(len(pred4)):
    pred_num.append(pred4[i].index(max(pred4[i])))
    if pred_num[i] == correct_num[i]:
        correct_pred += 1
print("Accuracy:",(correct_pred / len(pred4)) * 100)

Accuracy: 94.72222222222221


## b

In [19]:
layers5 = [(89, tanh()), (10, softmax())]
NN5 = NeuralNetwork(X_train, Y_train, layers5, cross_entropy_loss(), iteration = 801)
NN5.train()
pred5 = NN5.test(X_test)

iterations completed: 0
iterations completed: 100
iterations completed: 200
iterations completed: 300
iterations completed: 400
iterations completed: 500
iterations completed: 600
iterations completed: 700
iterations completed: 800


In [20]:
correct_num = Y_test.tolist()[0]
pred_num = []
correct_pred = 0
for i in range(len(pred5)):
    pred_num.append(pred5[i].index(max(pred5[i])))
    if pred_num[i] == correct_num[i]:
        correct_pred += 1
print("Accuracy:",(correct_pred / len(pred5)) * 100)

Accuracy: 97.5
